In [ ]:
!pip install gensim


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 577.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 12.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0

In [6]:
!pip install --upgrade numpy pandas gensim scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 111.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.


In [5]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
from google.colab import files
uploaded = files.upload()


Saving archive.zip to archive.zip


In [6]:
#load the dataset

df = pd.read_json("archive.zip",lines=True)
#read_json is used since the dataset is a JSON file
#lines=True is to tell that each line in the file is a separate object
sentences = df["headline"].unique().tolist()[:10000] #using only first 10k headlines to limit data size

#preprocessing function
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

nltk.download('punkt_tab')

def preprocess(text):
  tokens = word_tokenize(text.lower())
  tokens = [t for t in tokens if t not in stop_words]
  tokens = [lemmatizer.lemmatize(t) for t in tokens]
  return tokens

#tokenizing the sentences
tokenized_sentences = [preprocess(s) for s in sentences]

#creating the word2vec model
w2v_model = Word2Vec(tokenized_sentences)

#AvgWord2Vec

def avg_word2vec (tokens):
  #adding the vector of the word to the list only if the word is present in the vocabulary
  vectors = [w2v_model.wv[word] for word in tokens if word in w2v_model.wv]
  #calculating the mean along axis=0 (y axis) ==> the avg of all first elements, the avg of all second elements, etc from the vectors of fixed number of elements each
  return np.mean(vectors, axis=0) if vectors else np.zeros(w2v_model.vector_size)

#sentence vectors
sentence_vectors = [avg_word2vec(sentence) for sentence in tokenized_sentences]

#function to get the 5 most similar headlines

def get_top5_similar (input_sent):
  input_sent_tokens = preprocess(input_sent)
  input_sent_vector = avg_word2vec(input_sent_tokens)
  similarities = cosine_similarity([input_sent_vector],sentence_vectors)[0]
  #cosine similarity returns a 2d array that's why the [0]
  top5_indices = similarities.argsort()[-5:][::-1]
  #argsort returns the indices of the sentences as in the df in the ascending order
  return [sentences[i] for i in top5_indices]

input_sent = "The death of Pope"
top5 = get_top5_similar(input_sent)
for i in top5:
  print(i)




[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Protests Continue In U.S. After Deaths Of More Black Men
Teen Claims He Visited Heaven During Near Death Experience
How The U.S. Is Killing Off The Death Penalty
Worldwide Gun Deaths Reach 250,000 Yearly
Kansas Becomes Sixth State To Confirm Vaping-Related Death In National Outbreak
